# Creating and Valuing a CDS Contract vs Markit

Example CDS Valuation and comparison with market standard ISDA model on Markit website

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.finutils import *
from financepy.products.rates import *
from financepy.products.credit import *

####################################################################
# FINANCEPY BETA Version 0.185 - This build:  24 Oct 2020 at 20:12 #
#      This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy  #
#      Send any bug reports or comments to quant@financepy.com     #
####################################################################



## Creating a CDS Contract

This based on an example from Markit's CDS calculator website https://www.markit.com/markit.jsp?jsppage=pv.jsp

In [4]:
valuation_date = FinDate(21, 11, 2019)
settlement_date = valuation_date.addDays(1)
maturity_date = FinDate(20, 12, 2024)
cdsCoupon = 0.050
notional = ONE_MILLION
long_protection = True
tradeDate = FinDate(9, 8, 2019)

In [5]:
cds_contract = FinCDS(settlement_date, maturity_date, cdsCoupon, notional, long_protection)

In [6]:
print(cds_contract)

OBJECT TYPE: FinCDS
STEP-IN DATE: FRI 22 NOV 2019
MATURITY: FRI 20 DEC 2024
NOTIONAL: 1000000
RUNNING COUPON: 500.0bp
DAYCOUNT: DayCountTypes.ACT_360
FREQUENCY: FrequencyTypes.QUARTERLY
CALENDAR: CalendarTypes.WEEKEND
BUSDAYRULE: BusDayAdjustTypes.FOLLOWING
DATEGENRULE: DateGenRuleTypes.BACKWARD
ACCRUED DAYS: 63
PAYMENT_DATE, YEAR_FRAC, FLOW
FRI 20 SEP 2019,     0.000000,     0.000000
FRI 20 DEC 2019,     0.252778, 12638.888889
FRI 20 MAR 2020,     0.252778, 12638.888889
MON 22 JUN 2020,     0.261111, 13055.555556
MON 21 SEP 2020,     0.252778, 12638.888889
MON 21 DEC 2020,     0.252778, 12638.888889
MON 22 MAR 2021,     0.252778, 12638.888889
MON 21 JUN 2021,     0.252778, 12638.888889
MON 20 SEP 2021,     0.252778, 12638.888889
MON 20 DEC 2021,     0.252778, 12638.888889
MON 21 MAR 2022,     0.252778, 12638.888889
MON 20 JUN 2022,     0.252778, 12638.888889
TUE 20 SEP 2022,     0.255556, 12777.777778
TUE 20 DEC 2022,     0.252778, 12638.888889
MON 20 MAR 2023,     0.250000, 12500.000

## Build Ibor Curve

In [7]:
dcType = DayCountTypes.ACT_360
depo1 = FinIborDeposit(settlement_date, "1M", 0.017156, dcType)
depo2 = FinIborDeposit(settlement_date, "2M", 0.018335, dcType)
depo3 = FinIborDeposit(settlement_date, "3M", 0.018988, dcType)
depo4 = FinIborDeposit(settlement_date, "6M", 0.018911, dcType)
depo5 = FinIborDeposit(settlement_date, "12M", 0.019093, dcType)
depos = [depo1,depo2,depo3,depo4,depo5]

swapType = FinSwapTypes.PAY
dcType = DayCountTypes.THIRTY_E_360_ISDA
fixedFreq = FrequencyTypes.SEMI_ANNUAL
swap1 = FinIborSwap(settlement_date,"2Y",swapType,0.015630,fixedFreq,dcType)
swap2 = FinIborSwap(settlement_date,"3Y",swapType,0.015140,fixedFreq,dcType)
swap3 = FinIborSwap(settlement_date,"4Y",swapType,0.015065,fixedFreq,dcType)
swap4 = FinIborSwap(settlement_date,"5Y",swapType,0.015140,fixedFreq,dcType)
swap5 = FinIborSwap(settlement_date,"6Y",swapType,0.015270,fixedFreq,dcType)
swap6 = FinIborSwap(settlement_date,"7Y",swapType,0.015470,fixedFreq,dcType)
swap7 = FinIborSwap(settlement_date,"8Y",swapType,0.015720,fixedFreq,dcType)
swap8 = FinIborSwap(settlement_date,"9Y",swapType,0.016000,fixedFreq,dcType)
swap9 = FinIborSwap(settlement_date,"10Y",swapType,0.016285,fixedFreq,dcType)
swap10 = FinIborSwap(settlement_date,"12Y",swapType,0.01670,fixedFreq,dcType)
swaps = [swap1,swap2,swap3,swap4,swap5,swap6,swap7,swap8,swap9,swap10]

libor_curve = FinIborSingleCurve(valuation_date, depos, [], swaps)

Inserting synthetic deposit


# Build a CDS Curve

In [8]:
cdsFlatSpread = 0.0100

In [9]:
cds1 = FinCDS(settlement_date, "1Y", cdsFlatSpread)
cds2 = FinCDS(settlement_date, "2Y", cdsFlatSpread)
cds3 = FinCDS(settlement_date, "3Y", cdsFlatSpread)
cds4 = FinCDS(settlement_date, "4Y", cdsFlatSpread)
cds5 = FinCDS(settlement_date, "5Y", cdsFlatSpread)
cds6 = FinCDS(settlement_date, "7Y", cdsFlatSpread)
cds7 = FinCDS(settlement_date, "10Y", cdsFlatSpread)
cds8 = FinCDS(settlement_date, "15Y", cdsFlatSpread)

In [10]:
cdss = [cds1, cds2, cds3, cds4, cds5, cds6, cds7, cds8]

In [11]:
recovery_rate = 0.40

In [12]:
issuer_curve = FinCDSCurve(valuation_date, cdss, libor_curve, recovery_rate)

# Valuation Results

In [22]:
spd = cds_contract.parSpread(valuation_date, issuer_curve, recovery_rate) * 10000.0
print("FAIR CDS SPREAD %10.5f bp"% spd)

FAIR CDS SPREAD  100.00000 bp


In [23]:
v = cds_contract.value(valuation_date, issuer_curve, recovery_rate)
full_pv = v['full_pv']
clean_pv = v['clean_pv']

print("FULL VALUE  %12.2f"% full_pv)
print("CLEAN VALUE %12.2f"% clean_pv)

FULL VALUE    -198565.53
CLEAN VALUE   -189815.53


MARKIT CALCULATE -198,551 for the FULL VALUE

In [15]:
cleanp = cds_contract.clean_price(settlement_date, issuer_curve, recovery_rate)
print("CLEAN PRICE %12.6f"% cleanp)

CLEAN PRICE   118.983243


MARKIT CALCULATE 118.98%

In [16]:
accrued_days = cds_contract.accrued_days()
print("ACCRUED_DAYS", accrued_days)

ACCRUED_DAYS 63


MARKIT CALCULATE 63

In [17]:
accruedInterest = cds_contract.accruedInterest()
print("ACCRUED_COUPON", accruedInterest)

ACCRUED_COUPON -8750.0


MARKIT CALCULATE 8750

In [18]:
prot_pv = cds_contract.protectionLegPV(settlement_date, issuer_curve, recovery_rate)
print("PROTECTION_PV", prot_pv)

PROTECTION_PV 47458.06755888899


In [19]:
premPV = cds_contract.premiumLegPV(settlement_date, issuer_curve, recovery_rate)
print("PREMIUM_PV", premPV)

PREMIUM_PV 246040.91897641603


## Risk Measures

In [20]:
creditDV01 = cds_contract.creditDV01(settlement_date, issuer_curve, recovery_rate)
print("Credit DV01  %12.2f"% creditDV01)

Credit DV01        553.62


MARKIT FOUND 554

In [21]:
interestDV01 = cds_contract.interestDV01(settlement_date, issuer_curve, recovery_rate)
print("Interest DV01  %12.2f"% interestDV01)

Interest DV01         48.82


MARKIT FOUND 49

## Full Analysis 

In [24]:
cds_contract.printFlows(issuer_curve)

PAYMENT_DATE      YEAR_FRAC      FLOW           DF       SURV_PROB      NPV
FRI 20 DEC 2019   0.252778     12638.89     0.998620     0.998658     12604.51
FRI 20 MAR 2020   0.252778     12638.89     0.993728     0.994459     12490.03
MON 22 JUN 2020   0.261111     13055.56     0.988864     0.990140     12782.88
MON 21 SEP 2020   0.252778     12638.89     0.984127     0.985977     12263.85
MON 21 DEC 2020   0.252778     12638.89     0.980239     0.981832     12164.04
MON 22 MAR 2021   0.252778     12638.89     0.978222     0.977711     12088.06
MON 21 JUN 2021   0.252778     12638.89     0.975671     0.973607     12005.94
MON 20 SEP 2021   0.252778     12638.89     0.971923     0.969521     11909.61
MON 20 DEC 2021   0.252778     12638.89     0.968261     0.965451     11814.95
MON 21 MAR 2022   0.252778     12638.89     0.964776     0.961400     11723.02
MON 20 JUN 2022   0.252778     12638.89     0.961358     0.957365     11632.47
TUE 20 SEP 2022   0.255556     12777.78     0.958041   